## 컴파운드 패턴 개요

컴파운드 패턴은 2개 이상의 패턴을 합쳐 문제를 해결한다. 하지만 컴파운드 패턴은 단순히 여러 패턴의 조합이 아닌 문제를 해결하는 독립적인 솔루션이다.

## 모델-뷰-컨트롤러 패턴

MVC 패턴은 유저 인터페이스를 구현할 수 있는 유지보수가 용이한 디자인 패턴이다. MVC 패턴은 애플리케이션을 모델과 뷰, 컨트롤러로 나눠 구성한다. 각 파트는 맞물려 있으며 요청의 처리와 표현을 분리한다.

모델은 데이터와 비즈니스 로직(정보 저방 및 쿼리로직)을 처리하고 뷰는 데이터의 시각적 표현을 담당하며 컨트롤러는 사용자의 요청에 따라 모델과 뷰 사이에서 요청을 처리한다. 뷰와 컨트롤러는 모델에 의존하지만 그 반대는 아니다. 사용자가 데이터를 직접 요청하는 구조이기 때문이다. 모델의 독립성이 MVC 패턴의 중요한 부분이다.

In [ ]:
class Model(object):
    ser